# Data Cleaning for SET Data(Final)

Last Date: 07/10/2019

## Unit and Data Import

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [25]:
## Import data set
raw_filepath = 'SET data archive 3416 cases recd thru end of 03 2019.csv'

df = pd.read_csv(raw_filepath)
df.head()

,MERGE_DATE,archive_upload_2229,FULL_GRYD_ID_FB,Processing_status,Manual_notes_ETO,no_intake_question,no_retest1_question,no_retest2_question,FEEDBACK_STATUS,date_feedback_uploaded_ETO,...,othgpnorms_4bins,othgpcohe_8bins,othgpID_8bins,othgpemo_8bins,LN_CRIME,LN_VIOLENCE,SETDirections_ETO,SiteName_ETO,PROGRAM_NAME_ETO,enroll_ETO_num
0,2/8/2019,,77th 2-13-1494-1,2.0,"Missing response(s) S11a text, s11b, x10 and x15",,,,2.0,2/8/2019,...,1,3,5,5,0,0,If individual is still receiving services in 6...,1,2,1
1,1/16/2015,,RPARSA-2-64-1,9.0,,,,,0.0,,...,1,4,5,3,0,1.6094379124341,,,,1
2,4/28/2014,,RPARSA-2-77-1,9.0,,,,,0.0,,...,9,9,9,9,1.79175946922805,.693147180559945,,1,,1
3,3/18/2019,,Northeast-25-68-2,1.0,,,,,2.0,3/18/2019,...,9,9,9,9,1.09861228866811,1.38629436111989,If individual is still receiving services in 6...,1,14,1
4,3/18/2019,,Northeast-25-69-2,1.0,,,,,2.0,3/18/2019,...,9,9,9,9,.693147180559945,.693147180559945,If individual is still receiving services in 6...,1,14,1


In [26]:
df.shape

(3416, 432)

## Investigating Data

### Drop the wrong version 

In [27]:
df = df[df['SET_version_num'] == 3]
df['SET_version_num'].value_counts()

3    2800
Name: SET_version_num, dtype: int64

In [28]:
df.head()

,MERGE_DATE,archive_upload_2229,FULL_GRYD_ID_FB,Processing_status,Manual_notes_ETO,no_intake_question,no_retest1_question,no_retest2_question,FEEDBACK_STATUS,date_feedback_uploaded_ETO,...,othgpnorms_4bins,othgpcohe_8bins,othgpID_8bins,othgpemo_8bins,LN_CRIME,LN_VIOLENCE,SETDirections_ETO,SiteName_ETO,PROGRAM_NAME_ETO,enroll_ETO_num
0,2/8/2019,,77th 2-13-1494-1,2.0,"Missing response(s) S11a text, s11b, x10 and x15",,,,2.0,2/8/2019,...,1,3,5,5,0,0,If individual is still receiving services in 6...,1,2,1
3,3/18/2019,,Northeast-25-68-2,1.0,,,,,2.0,3/18/2019,...,9,9,9,9,1.09861228866811,1.38629436111989,If individual is still receiving services in 6...,1,14,1
4,3/18/2019,,Northeast-25-69-2,1.0,,,,,2.0,3/18/2019,...,9,9,9,9,.693147180559945,.693147180559945,If individual is still receiving services in 6...,1,14,1
5,3/18/2019,,Northeast-25-70-2,1.0,,,,,2.0,3/18/2019,...,9,9,9,9,1.09861228866811,.693147180559945,If individual is still receiving services in 6...,1,14,1
6,4/14/2017,1,Olympic-12-39-2,3.0,ETO continues to show duplicate GRYD IDs - the...,2,,,2.0,6/8/2018,...,1,5,2,3,1.38629436111989,1.6094379124341,If individual is still receiving services in 6...,1,17,2


### Creating Unique ID

In [29]:
#showing how many ETO_IDs are missing

df['ETO_ID_FB'] = pd.to_numeric(df['ETO_ID_FB'], downcast = 'integer', errors = 'coerce')
df['ETO_ID_FB'].isnull().value_counts()

#None of them are missing

False    2800
Name: ETO_ID_FB, dtype: int64

In [30]:
# Change the column name to UniqueID
df.rename(columns={'ETO_ID_FB':'UniqueID'}, inplace = True)

### Column Processing_status

In [31]:
#Drop incomplete processing status
df = df[df['Processing_status'] < 2]

In [32]:
df.head()
df['Processing_status'].value_counts()

1.5    1646
1.0    1123
Name: Processing_status, dtype: int64

## Questionnaire part

### Family Section (clean data for no family people )

In [11]:
#list the index for those who don't have family
df['no_fam_num'] = pd.to_numeric(df['no_fam_num'], downcast = 'integer', errors = 'coerce')
NoFamppl = df[df['no_fam_num'] == 0].index.tolist()
NoFamppl.extend(df[df['no_fam_num'].isnull()].index.tolist())

#list the questions about family
f_que = df.loc[:,'F1':'F3'].columns.tolist()
f_que.extend(df.loc[:,'FS90_younger':'FS90_older_adults'].columns.tolist())

#convert these values to numeric
for col in f_que:
    df[col] = pd.to_numeric(df[col], downcast = 'integer', errors = 'coerce')


In [12]:
#change their responses from f1 to f3 to 0
for index in NoFamppl:
    for col in f_que:
        df.loc[index, col] = 0

In [13]:
#list the additional questions about family
f_que2 = df.loc[:,'F4':'FS98'].columns.tolist()
f_que2.extend(df.loc[:,'FS100':'F17x'].columns.tolist())
f_que2.extend(df.loc[:,'F18':'F31b'].columns.tolist())

#convert these values to numeric
for col in f_que2:
    df[col] = pd.to_numeric(df[col], downcast = 'integer', errors = 'coerce')

In [14]:
#change their responses from f4 to f31b to nan
for index in NoFamppl:
    for col in f_que:
        df.loc[index, col] = np.nan

### Group Section (clean data for no group people )

In [15]:
#list the index for those who don't have group
df['no_group_num'] = pd.to_numeric(df['no_group_num'], downcast = 'integer', errors = 'coerce')
NoGrppl = df[df['no_group_num'] == 0].index.tolist()
NoGrppl.extend(df[df['no_group_num'].isnull()].index.tolist())

#list the group questions
g_que = df.loc[:,'G2':'G9'].columns.tolist()
g_que.extend(df.loc[:,'G10x':'G23x'].columns.tolist())
g_que.extend(df.loc[:,'G24':'G37'].columns.tolist())

#converting to numeric
for col in g_que:
    df[col] = pd.to_numeric(df[col], downcast = 'integer', errors = 'coerce')
    
#change their responses from G2 to G37 to nan
for index in NoGrppl:
    for col in g_que:
        df.loc[index, col] = np.nan

### Other Group (O) Section (clean data for no other group people)

In [16]:
#list the index for those who don't have other group
df['no_othgp_num'] = pd.to_numeric(df['no_othgp_num'], downcast = 'integer', errors = 'coerce')
No_Other = df[df['no_othgp_num'] != 1].index.tolist()

#list the other group questions
o_que = df.loc[:,'O4':'O9'].columns.tolist()
o_que.extend(df.loc[:,'O10':'O15'].columns.tolist())
o_que.extend(df.loc[:,'O16':'O24'].columns.tolist())

#convert to numeric
for col in o_que:
    df[col] = pd.to_numeric(df[col], downcast = 'integer', errors = 'coerce')
        
#change their responses from O4 to O9 to nan
for index in No_Other:
    for col in o_que:
        df.loc[index, col] = np.nan

### Converting numbers to Nan

In [17]:
# S section
que = df.loc[:,'S1':'S3'].columns.tolist(){}
que.extend(df.loc[:,'S4x':'S6ax'].columns.tolist())
que.extend(df.loc[:,'S9':'S10'].columns.tolist())
que.extend(df.loc[:,'S10x':'S11a'].columns.tolist())
que.extend(df.loc[:,'S11b':'S16'].columns.tolist())

# ME,P,C section
que.extend(df.loc[:,'ME17':'C5b'].columns.tolist())

# FS section
que.extend(df.loc[:,'FS89_foster_now':'FS90_older_adults'].columns.tolist())

# F section
que.extend(f_que)

# G section
que.extend(g_que)

# X section
que.extend(df.loc[:,'X18':'X22'].columns.tolist())
que.extend(df.loc[:,'X10':'X17'].columns.tolist())

# A section
que.extend(['A1'])
que.extend(['A2'])
que.extend(['A3'])
que.extend(['A4'])

#O Section
que.extend(o_que)

# IC section
que.extend(df.loc[:,'IC1':'IC7'].columns.tolist())

# CIW section
que.extend(df.loc[:,'CIW1':'CIW5'].columns.tolist())

In [18]:
print(que)

['S1', 'S2', 'S3', 'S4x', 'S5x', 'S7x', 'S6ax', 'S9', 'S10', 'S10x', 'S10z', 'S11a', 'S11b', 'S12', 'S13', 'S14', 'S16', 'ME17', 'ME18', 'ME19', 'ME20a', 'ME20b', 'ME20x', 'ME22_ER', 'P22', 'P23', 'P24', 'P25', 'P26', 'C1a', 'C1b', 'C2a', 'C2b', 'C3a', 'C3b', 'C4a', 'C4b', 'C5a', 'C5b', 'FS89_foster_now', 'FS89_foster_past', 'FS89_group_now', 'FS89_group_past', 'FS89_adopted_now', 'FS89_adopted_past', 'FS89_parents_now', 'FS89_parents_past', 'FS89_relatives_now', 'FS89_relatives_past', 'FS89_spouse_now', 'FS89_spouse_past', 'FS89_boygirlfriend_now', 'FS89_boygirlfriend_past', 'FS89_alone_now', 'FS89_alone_past', 'FS90_younger', 'FS90_teenagers', 'FS90_20_29_adults', 'FS90_30_49_adults', 'FS90_older_adults', 'F1', 'F2', 'F3', 'FS90_younger', 'FS90_teenagers', 'FS90_20_29_adults', 'FS90_30_49_adults', 'FS90_older_adults', 'G2', 'G3a', 'G3b', 'EG1', 'EG2_most', 'EG2_youngest', 'EG2_oldest', 'EG3', 'EG4x', 'G5', 'G5_num', 'G6', 'G7', 'G8', 'G9', 'G10x', 'G11', 'G12x', 'G13', 'G14x', 'G15',

In [19]:
for col in que:
    df[col].replace('System',np.NaN)  
    df[col] = pd.to_numeric(df[col], downcast = 'integer', errors = 'coerce')
    df.loc[(df[col] == 999), col] = np.nan
    df.loc[(df[col] == 888), col] = np.nan
    df.loc[(df[col] == 777), col] = np.nan
    df.loc[(df[col] == 555), col] = np.nan

## Create new dataframe of full questionnaire and columns we are unsure about

#### Continued-July8,2019

In [20]:
#list all columns needed to kept
col_kept = ['UniqueID']
col_kept.extend(df.loc[:,'no_intake_question':'no_retest2_question'].columns.tolist())
col_kept.extend(df.loc[:,'SET_program_num':'baseline_type'].columns.tolist())
col_kept.extend(['enroll_SET_type_num'])
col_kept.extend(['enroll_SET_type'])
col_kept.extend(df.loc[:,'cycle_1_1':'retest'].columns.tolist())
col_kept.extend(df.loc[:,'GRYD_Zone_num':'INTERVIEW_DATE_FB'].columns.tolist())
col_kept.extend(df.loc[:,'ENROLLMENT_DATE_1':'SEX_1'].columns.tolist())
col_kept.extend(df.loc[:,'GROUP_POSITION_1':'group_position'].columns.tolist())
col_kept.extend(df.loc[:,'TATTOO_REMOVE_1':'comments_page1_1'].columns.tolist())

#beginning of questionnaire
col_kept.extend(df.loc[:,'S1':'S16'].columns.tolist())
col_kept.extend(df.loc[:,'ME17':'C5b'].columns.tolist())
col_kept.extend(df.loc[:,'Group_name_part1':'FS90_older_adults'].columns.tolist())
col_kept.extend(df.loc[:,'F1':'FS98'].columns.tolist())
col_kept.extend(df.loc[:,'FS100':'F17x'].columns.tolist())
col_kept.extend(df.loc[:,'F18':'F31b'].columns.tolist())
col_kept.extend(['group_name_part2'])
col_kept.extend(df.loc[:,'no_group_num':'X11_FIXED'].columns.tolist())
col_kept.extend(df.loc[:,'G10x':'G23x'].columns.tolist())
col_kept.extend(df.loc[:,'G24':'G37'].columns.tolist())
col_kept.extend(df.loc[:,'X18':'X22'].columns.tolist())
col_kept.extend(df.loc[:,'A1':'no_othgp_num'].columns.tolist())
col_kept.extend(df.loc[:,'O4':'O15'].columns.tolist())
col_kept.extend(df.loc[:,'O16':'O24'].columns.tolist())
col_kept.extend(df.loc[:,'X10':'X17'].columns.tolist())
#end of questionnaire

col_kept.extend(df.loc[:,'IC1':'IC8_Not_Honest'].columns.tolist())
col_kept.extend(df.loc[:,'Completed_by_CIW':'CIW6_COMMENTS'].columns.tolist())

In [21]:
#make a general dataframe for asking questions 
df_new = df.filter(col_kept, axis=1)
df_new.head()

,UniqueID,no_intake_question,no_retest1_question,no_retest2_question,SET_program_num,ETO_PROGRAM,SET_admin_num,baseline_type,enroll_SET_type_num,enroll_SET_type,...,Completed_by_CIW,CIW_CLIENT_ID,CIW_INTERVIEW_DATE,CIW_COMPLETED_DATE,CIW1,CIW2,CIW3,CIW4,CIW5,CIW6_COMMENTS
3,49866,,,,1,1,1,,,,...,999,,,,1.0,4.0,4.0,1.0,3.0,Client's son is helping him to change little b...
4,49868,,,,1,1,1,,,,...,999,,,,2.0,3.0,4.0,1.0,3.0,Slow progress
5,49870,,,,1,1,1,,,,...,999,,,,2.0,3.0,3.0,1.0,3.0,She goes there a lot ?home? She is trying to c...
8,47002,,,,1,1,1,,,,...,999,Southwest 2-23-230-1,9/13/2018,9/13/2018,2.0,3.0,1.0,4.0,0.0,Client has been attending program and is stayi...
13,41570,,,,2,2,1,,1,intake,...,999,SSSEARE-0-100-1,,,NaN,NaN,NaN,NaN,NaN,no page 17 completed


In [22]:
df_new.shape

(2769, 253)

In [23]:
fpath = 'SET data archive 3416 cases recd thru end of 03 2019_cleaned_general.csv'
df_new.to_csv(fpath)